In [1]:
from random import randint
from math import ceil
from Crypto.Cipher import AES
CODING_TABLE = "ABCDEFGHIJKLMNOPQRSTUVWXYZ234567".lower()
SECRET_KEY = b'Sixteen byte key'
IV_128 = b'0123456789ABCDEF'

# Max. DNS length

    DNS maximium length of label = 63 chars
    DNS maximum length of whole string = 253 chars
    253 - .dnsr.uk.to (11) = 242 chars
    (63 + .) * 3 + 50 = 242 chars
    encoding 63 * 3 + 50 = 239 useful chars
    
__Maximum length request (239 chars) tested working is below__

In [38]:
s = ''
for i in range(50):
    s += CODING_TABLE[randint(0, len(CODING_TABLE) - 2)]
s += '.'
for j in range(3):
    for i in range(63):
        s += CODING_TABLE[randint(0, len(CODING_TABLE) - 2)]
    s += '.'
s += 'dnsr.uk.to'
print(len(s))
print(s)

253
ryjznzmpnexedpcnxnicuni4gf6bcpcy235kovmg4xca2qfakl.rqpaqhigl553n5rhjhuvmsixwzohwjzgnjxqota22cupqjnrsuhmybxapgz5mgp.cazd4szpsaltbvjrluzuejawvvfo44l2eszmzswievw4aboasynxtjc2o62e3gs.mtenhctf4t5b2axpyr34opfkfski3ezeim5rjw4w6ujurcw4e6v5olt2vk5pbuk.dnsr.uk.to


base32 encodes 5 bit into 1 char, so we have 239 * 5 bits payload = 1195 bits = __149 bytes max__

from which we take 3 bytes for CRC + len

__146 byetes user payload__

# Encode / decode base32 

In [2]:
# for B32
DECODING_TABLE = {}
for i,c in enumerate(CODING_TABLE.lower()):
    DECODING_TABLE[c] = i

def enc_b32(pl):
    chars = ''
    res = int.from_bytes(pl, 'little')
    nBits = len(pl) * 8
    while nBits > 0:
        chars += CODING_TABLE[res & 0x1F]
        res >>= 5
        nBits -= 5
    return chars

def dec_b32(chars):
    res = 0
    for i, c in enumerate(chars.lower()):
        res |= DECODING_TABLE[c] << (i * 5)
    return res.to_bytes(ceil(len(chars) * 5 / 8), 'little')

c = enc_b32(b'\x01\x02\x03ABCDEFG\xFF\xF1\x00')
print(c)
print(dec_b32(c))

bqagqajidcrked5i7p4ba
b'\x01\x02\x03ABCDEFG\xff\xf1\x00\x00'


# CRC16

In [3]:
class Crc16:    
    def __init__(self):
        ''' Modbbus compatible '''
        self._resetRunnningCRC()
        
    def _resetRunnningCRC(self):
        self.c = 0xFFFF
        
    def _runningCRC(self, inputByte):
        self.c ^= inputByte
        self.c &= 0xFFFF
        for b in range(8):  # For each bit in the byte
            if self.c & 1:
                self.c = (self.c >> 1) ^ 0xA001
            else: 
                self.c = (self.c >> 1)
            self.c &= 0xFFFF
            
    def getCrc( self, dataBytes ):
        self._resetRunnningCRC()
        for b in dataBytes:
            self._runningCRC( b )
        return self.c.to_bytes(2, 'big')

crc = Crc16()

# Encrypt

In [4]:
def hprint(bs):
    print()
    for b in bs:
        print('{:02x}'.format(b), end=' ')
    print()

In [42]:
from Crypto.Cipher import AES
from Crypto import Random

# IV_128 = Random.new().read(AES.block_size)

cipher = AES.new(SECRET_KEY, AES.MODE_CBC, IV_128)
cipherd = AES.new(SECRET_KEY, AES.MODE_CBC, Random.new().read(AES.block_size))

def dns_enc(pl):
    # Add CRC16 fields
    pl = bytes(pl) + crc.getCrc(pl)
    
    # Pad pl to mod. 16 bytes
    n_pad = 16 - (len(pl) % 16)
    pl += bytes([n_pad] * n_pad)
    
    # AES encrypt
    msg = cipher.encrypt(pl)
    hprint(msg)
    
    # base32 encode and add some . every 63 characters
    msg32 = enc_b32(msg)
    LBL_LEN = 63
    msg32 = '.'.join([msg32[i: i + LBL_LEN] for i in range(0, len(msg32), LBL_LEN)])
    return msg32

def dns_dec(msg32):
    # remove '.', base32 decode
    msg32 = msg32.replace('.', '').upper()
    msg = dec_b32(msg32)
    
    # msg must be a integer multiple of 16 bytes long, 
    # clip the eventual \x00 at the end
    msg = msg[:len(msg) - (len(msg) % 16)]
    
    # AES decrypt
    pl = cipherd.decrypt(msg)
    
    # remove padding
    pl = pl[: len(pl) - pl[-1]]
    
    # check crc
    crc_r = pl[-2:]
    pl = pl[:-2]
    if crc.getCrc(pl) != crc_r:
        raise RuntimeError("CRC ERR")
    return pl

txt = b'12345abcABC'
for i in range(4):
    c_txt = dns_enc(txt)
    print(c_txt)


11 00 db e2 a7 1d 43 ef 27 ea eb bb eb a6 a5 3f 
raawnr7u5yq66ti5l7oxotwu7b

6e d4 ec 9a bc 37 1b 14 e9 8b d8 d8 a2 01 be 94 
odvzonsxxzgiruprygwf2ayxue

fc d4 dd 8f 69 9c 13 82 3a fe a4 f3 83 e5 f7 5c 
4hv35hgn44eei5y7e54hys764c

e2 b9 c0 e1 e0 78 12 7e b2 7e 69 6e 6a 13 f2 5b 
cpob4qd4yte4hz2pjt3uwji63c


# Stress test encrypt and decrypt

In [ ]:
i = 0
while(True):
    pl = bytes([randint(0, 255) for x in range(randint(0, 255))])
    c_txt = dns_enc(pl)
    d_pl = dns_dec(c_txt)
    if d_pl != pl:
        print('error')
        break
    i += 1

In [700]:
print(i)

1160304


# Try the class

In [2]:
from dns_coder import DnsCoder
dc = DnsCoder(SECRET_KEY, CODING_TABLE)

In [15]:
txt = b'12345abcABC'
c_txt = dc.dns_enc(txt)
print(c_txt)
dc.dns_dec(c_txt)

c6mzliuwsnlswbhrxbn44cdqze


b'12345abcABC'

In [11]:
test_ = test.replace(dpf, '')
print(test_)
dnsc.dns_dec(test_)

tmbqdetkqf-jb3b-rr-vu727uvd6vekefuerpk8i71ibprztgckt..


RuntimeError: CRC ERR